In [1]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
model = fasterrcnn_resnet50_fpn_v2(pretrained=True, progress=True)

model.eval()


In [ ]:
import torchvision.transforms as transforms
from PIL import Image

image = Image.open("../image.png")
# transform input image to tensor
transform = transforms.Compose([transforms.ToTensor()])

image_tensor = transform(image)
image_tensor

In [ ]:
image_tensor.shape

In [ ]:
image_tensor = image_tensor.unsqueeze(dim=0)
image_tensor.shape

In [ ]:
predictions = model(image_tensor)
predictions

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

classNames = ["PMF 1", "PMN 2", "F1", "RGD 5", "RKG 3", "TM 62", "OZM 72", "MON 50"]
# Set a threshold for displaying predictions
CONFIDENCE_THRESHOLD = 0.5  # for example, 50%

# Convert tensor image back to numpy for visualization
img = image_tensor.squeeze().detach().cpu().numpy()  # Adjust dimensions as necessary
img = img.transpose(1, 2, 0)  # Rearrange the dimensions from CxHxW to HxWxC for image display

# Create figure and axes
fig, ax = plt.subplots(1, figsize=(12, 9))

# Display the image
ax.imshow(img)
preds = predictions
# Add bounding boxes and labels
for box, label, score in zip(preds[0]['boxes'], preds[0]['labels'], preds[0]['scores']):
    if score.item() > CONFIDENCE_THRESHOLD:
        x1, y1, x2, y2 = box.detach().cpu().numpy()  # Make sure to detach and convert boxes as well

        # Adjust for potential off-by-one error
        label_index = label.item()  # Assuming model's labels are 0-indexed
        # If labels are 1-indexed, subtract 1: label_index = label.item() - 1

        label_name = classNames[label_index]
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(x1, y1, f'{label_name} {score.item():.2f}', color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

plt.axis('off')  # to turn off axes for better visualization
plt.show()


In [ ]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from pytorch_detection_tools import get_predictions, visualize_predictions

# Load the pretrained model
model = fasterrcnn_resnet50_fpn_v2(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 9)
model_state_path = '../trained_models/exp_faster_rcnn.pth'
# Load the saved model weights'
checkpoint = torch.load(model_state_path)
model.load_state_dict(checkpoint['model_state_dict'])
# evaluation mode
model.eval()
# Move the model to the appropriate device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
image_path = "../pmn2_52.jpg"
boxes, labels, scores = get_predictions(model, image_path, device)
visualize_predictions(image_path, boxes, labels, scores)


In [ ]:
from training_tools import test, collate_fn, get_custom_dataset
import torch 

print("Loading dataset...")
train_data_dir = "../explosives_dataset/coco/test"
my_dataset = get_custom_dataset(train_data_dir)
print('Number of samples: ', len(my_dataset))
# DataLoader instance
data_loader = torch.utils.data.DataLoader(my_dataset,
                         batch_size=2,
                         shuffle=True,
                         collate_fn=collate_fn)

model = fasterrcnn_resnet50_fpn_v2(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 9)
model_state_path = '../trained_models/exp_faster_rcnn.pth'
# Load the saved model weights'
checkpoint = torch.load(model_state_path)
model.load_state_dict(checkpoint['model_state_dict'])
test(model, data_loader)

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from training_tools import collate_fn, get_custom_dataset, save_model, create_ssd_model
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model_state_path = './trained_models/exp_faster_rcnn.pth'
print("Loading dataset...")
train_data_dir = "../explosives_small_dataset/coco/train"

my_dataset = get_custom_dataset(train_data_dir)
print('Number of samples: ', len(my_dataset))
# DataLoader instance
data_loader = DataLoader(my_dataset,
                         batch_size=2,
                         shuffle=True,
                         collate_fn=collate_fn)

# Initialize the model
print("Loading the model...")
num_classes = 9
model = fasterrcnn_resnet50_fpn_v2(pretrained=True)
# Update the classifier head
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params=params, lr=0.00001, momentum=0.9)
lr_scheduler = StepLR(optimizer, step_size=2)
epoch = 0
model.train()
i = 0
for images, targets in data_loader:
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    optimizer.zero_grad()
    losses = model(images, targets)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    if not i % 2:
        print(f"Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}")
    i += 1
lr_scheduler.step()
epoch+=1
# save_model(model, model_state_path, optimizer, epoch, loss)